# 🎯 Qiskit Certification - Exam Practice Notebook

> **Purpose**: Direct, runnable code examples for every exam concept  
> **Based on**: EXAM_REVISION_MASTER.md  
> **Pattern**: Each concept appears exactly ONCE with practice code

---

## 📋 Quick Navigation

| Section | Weight | Key Topics |
|---------|--------|------------|
| **§1** | 16% | Gates (X,Y,Z,H,S,T), Multi-qubit (CX,CZ,SWAP), State Prep |
| **§2** | 11% | Visualization (circuit.draw, plot_histogram, plot_bloch) |
| **§3** | 18% | Circuit Creation (compose, barrier, parameters, library) |
| **§4** | 15% | Transpilation (optimization_level, backend, target) |
| **§5** | 12% | Sampler (result[0].data.meas.get_counts()) |
| **§6** | 12% | Estimator (SparsePauliOp, expectation values) |
| **§7** | 10% | Results (RIDMG pattern extraction) |
| **§8** | 6% | OpenQASM (from_qasm_str, qasm3_import) |
| **§9** | 3% | Quantum Info (Clifford, Operator, Statevector, Fidelity) |

---

## ⚡ Critical Patterns - MEMORIZE THESE

```python
# Properties (NO parentheses)
qc.num_qubits, qc.num_clbits, qc.data, qc.parameters

# Methods (WITH parentheses)  
qc.depth(), qc.size(), qc.count_ops(), qc.draw()

# Sampler Result
counts = result[0].data.meas.get_counts()

# Estimator Result
expectation = result[0].data.evs

# OpenQASM Import (STATIC method)
qc = QuantumCircuit.from_qasm_str(qasm_string)  # NOT qc.from_qasm_str()

# Operator Equivalence
op1.equiv(op2)  # Use .equiv() not ==

# Clifford Gates
{H, S, S†, CNOT, X, Y, Z}  # T is NOT Clifford!
```

In [ ]:
# Setup - Import Everything Once
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import QFT, RealAmplitudes, EfficientSU2, TwoLocal
from qiskit.primitives import StatevectorSampler as Sampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator, state_fidelity
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_state_qsphere
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
print("✅ All imports successful - Ready for exam practice!")

In [ ]:
import numpy as np
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Pauli

# Define the matrix as a NumPy array (representing the ZZ operator for 2 qubits)
zz_matrix = np.array([
    [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
    [0.+0.j, -1.+0.j, 0.+0.j, 0.+0.j],
    [0.+0.j, 0.+0.j, -1.+0.j, 0.+0.j],
    [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]
])

# === MATHEMATICAL ANALYSIS TO IDENTIFY THE OPERATOR ===
print("=" * 70)
print("HOW TO IDENTIFY OPERATORS FROM MATRICES MATHEMATICALLY")
print("=" * 70)

print("\n📊 STEP 1: Analyze Matrix Structure")
print("-" * 70)
print("Given matrix:")
print(zz_matrix)
print("\nKey observations:")
print("  • Diagonal matrix (all off-diagonal elements = 0)")
print("  • Elements are ±1 (real, no imaginary parts)")
print("  • Size: 4×4 → operates on 2 qubits (2² = 4 basis states)")

print("\n📊 STEP 2: Map Diagonal Elements to Basis States")
print("-" * 70)
print("Diagonal elements correspond to computational basis states:")
print("  Position 0 (|00⟩): +1")
print("  Position 1 (|01⟩): -1")
print("  Position 2 (|10⟩): -1")
print("  Position 3 (|11⟩): +1")

print("\n📊 STEP 3: Identify Pattern")
print("-" * 70)
print("The eigenvalues are +1 or -1 based on parity:")
print("  • +1 when both qubits have SAME value (|00⟩, |11⟩)")
print("  • -1 when qubits have DIFFERENT values (|01⟩, |10⟩)")
print("\nThis is the signature of Z⊗Z (ZZ operator)!")

print("\n📊 STEP 4: Verify with Tensor Product")
print("-" * 70)
print("Z gate matrix:")
z_matrix = np.array([[1, 0], [0, -1]])
print(z_matrix)
print("\nZ⊗Z = Kronecker product of Z with itself:")
zz_computed = np.kron(z_matrix, z_matrix)
print(zz_computed)
print(f"\nMatrices match: {np.allclose(zz_matrix, zz_computed)}")

print("\n📊 STEP 5: General Rules for Identification")
print("-" * 70)
print("Common patterns:")
print("\n1. SINGLE-QUBIT GATES (2×2 matrices):")
print("   • X: [[0, 1], [1, 0]] → Bit flip")
print("   • Z: [[1, 0], [0, -1]] → Phase flip on |1⟩")
print("   • H: [[1, 1], [1, -1]]/√2 → Symmetric, creates superposition")
print("   • Y: [[0, -i], [i, 0]] → Bit + phase flip")

print("\n2. TWO-QUBIT PAULI TENSOR PRODUCTS (4×4 matrices):")
print("   • Tensor product A⊗B means: Apply B to qubit 0, A to qubit 1")
print("   • Formula: (A⊗B)[i·n+j, k·n+l] = A[i,k] · B[j,l]")
print("\n   ZZ (diagonal): [+1, -1, -1, +1]")
print("     • ZZ|00⟩ = Z|0⟩⊗Z|0⟩ = (+1)⊗(+1)|00⟩ = +1|00⟩")
print("     • ZZ|01⟩ = Z|0⟩⊗Z|1⟩ = (+1)⊗(-1)|01⟩ = -1|01⟩")
print("     • ZZ|10⟩ = Z|1⟩⊗Z|0⟩ = (-1)⊗(+1)|10⟩ = -1|10⟩")
print("     • ZZ|11⟩ = Z|1⟩⊗Z|1⟩ = (-1)⊗(-1)|11⟩ = +1|11⟩")
print("\n   XX (anti-diagonal, swaps |01⟩↔|10⟩):")
print("     • XX|00⟩ = X|0⟩⊗X|0⟩ = |1⟩⊗|1⟩ = |11⟩")
print("     • XX|01⟩ = X|0⟩⊗X|1⟩ = |1⟩⊗|0⟩ = |10⟩")
print("     • XX|10⟩ = X|1⟩⊗X|0⟩ = |0⟩⊗|1⟩ = |01⟩")
print("     • XX|11⟩ = X|1⟩⊗X|1⟩ = |0⟩⊗|0⟩ = |00⟩")
print("\n   IZ (only affects qubit 0): [+1, -1, +1, -1]")
print("     • IZ|00⟩ = I|0⟩⊗Z|0⟩ = |0⟩⊗(+1)|0⟩ = +1|00⟩")
print("     • IZ|01⟩ = I|0⟩⊗Z|1⟩ = |0⟩⊗(-1)|1⟩ = -1|01⟩")
print("     • IZ|10⟩ = I|1⟩⊗Z|0⟩ = |1⟩⊗(+1)|0⟩ = +1|10⟩")
print("     • IZ|11⟩ = I|1⟩⊗Z|1⟩ = |1⟩⊗(-1)|1⟩ = -1|11⟩")

print("\n3. CONTROLLED GATES (4×4 with block structure):")
print("   • CNOT: [[I, 0], [0, X]] → Identity on |0⟩, X on |1⟩")
print("   • CZ: [[I, 0], [0, Z]] → Diagonal with -1 at |11⟩")

print("\n4. KEY PROPERTIES TO CHECK:")
print("   • Unitary: U†U = I (preserves probability)")
print("   • Hermitian: U† = U (observable/measurement)")
print("   • Eigenvalues: ±1 for Pauli, complex for rotations")
print("   • Trace: Sum of diagonal → symmetry properties")



In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.measure(0, 0)
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.h(1)
with else_:
    qc.x(1)
qc.measure(1, 1)

qc.draw('mpl')

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
state = Statevector(qc)
plot_state_qsphere(state)

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import CouplingMap

# Create circuit to transpile
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Create pass manager with specific constraints

coupling_map = CouplingMap([[0,1], [1,2]])

pass_manager = generate_preset_pass_manager(
    optimization_level=3,
    coupling_map=coupling_map,
    basis_gates=['h', 'swap', 'cx'],
    initial_layout=[0, 2]
)

# Apply transpilation
tqc = pass_manager.run(qc)

# Visualize the original vs transpiled circuit
print("=" * 70)
print("ORIGINAL CIRCUIT")
print("=" * 70)
print(f"Depth: {qc.depth()}, Gates: {qc.count_ops()}")
qc.draw('mpl')

print("\n" + "=" * 70)
print("TRANSPILED CIRCUIT - Parameter Explanation")
print("=" * 70)

print("\n📋 Pass Manager Parameters:")
print("-" * 70)
print("1. optimization_level=3")
print("   → Highest optimization (0-3)")
print("   → Aggressively reduces gate count and depth")
print("   → Uses advanced synthesis and cancellation")

print("\n2. coupling_map=[[0,1], [1,2]]")
print("   → Defines allowed qubit connections:")
print("   → 0 ↔ 1 ↔ 2 (linear chain)")
print("   → Gates can only act on connected qubits")
print("   → SWAP gates inserted if needed for connectivity")

print("\n3. basis_gates=['h', 'swap', 'cx']")
print("   → Target hardware only supports these gates")
print("   → All gates decomposed into: H, SWAP, CX only")
print("   → Other gates (like Rz, U) would be decomposed")

print("\n4. initial_layout=[0,2]")
print("   → Maps logical qubits to physical qubits:")
print("   → Logical qubit 0 → Physical qubit 0")
print("   → Logical qubit 1 → Physical qubit 2")
print("   → Skips physical qubit 1!")
print("   → ⚠️ PROBLEM: Qubits 0 and 2 are NOT connected!")

print("\n⚠️  ROUTING ISSUE:")
print("-" * 70)
print("Coupling map: 0 ↔ 1 ↔ 2")
print("Initial layout places qubits on 0 and 2 (not adjacent!)")
print("To execute CX(0,1) logically:")
print("  → Physical qubits 0 and 2 need connection")
print("  → But they're not directly connected")
print("  → Transpiler must insert SWAP gates via qubit 1")

print("\n" + "=" * 70)
print("TRANSPILED RESULT")
print("=" * 70)
print(f"Depth: {tqc.depth()}, Gates: {tqc.count_ops()}")
print(f"Layout: {tqc.layout}")
tqc.draw('mpl')

print("\n🔍 What Happened:")
print("-" * 70)
print("1. H gate on logical q0 → Applied to physical q0 (allowed)")
print("2. CX(logical q0, logical q1) needs CX(physical q0, physical q2)")
print("3. But q0 and q2 are NOT connected in coupling map")
print("4. Transpiler routes through q1 using SWAP gates:")
print("   → SWAP(1,2) moves q1 from physical 2 to physical 1")
print("   → Now CX(0,1) can execute (q0 and q1 are connected)")
print("   → May need another SWAP to restore original layout")


In [ ]:
from qiskit.primitives import StatevectorEstimator as Estimator

# Precision and Shots Relationship in Quantum Computing
print("1. StatevectorEstimator (what we imported)")
print("   • Uses exact statevector simulation")
print("   • NO shots - computes expectation values analytically")
print("   • NO statistical noise")
print("   • Precision parameter is ignored (always exact)")
print("   • Used for: Testing, small circuits, exact results")

print("\n2. Hardware/Runtime Estimator (qiskit_ibm_runtime)")
print("   • Runs on real quantum hardware or noisy simulators")
print("   • DOES use shots for sampling")
print("   • HAS statistical noise")
print("   • Precision parameter controls shot allocation")
print("   • Used for: Real quantum experiments")

print("=" * 70)
print("HOW PRECISION AFFECTS NUMBER OF SHOTS")
print("=" * 70)

print("\n📊 FUNDAMENTAL RELATIONSHIP:")
print("-" * 70)
print("Standard Error = 1 / √(shots)")
print("Precision ∝ 1 / Standard Error")
print("Therefore: Precision ∝ √(shots)")
print("\nTo improve precision by factor of N:")
print("  → Need N² times more shots")

print("\n🔢 MATHEMATICAL EXPLANATION:")
print("-" * 70)
print("For a measurement outcome with probability p:")
print("  • Standard deviation: σ = √(p(1-p))")
print("  • Standard error: SE = σ/√(shots) = √(p(1-p))/√(shots)")
print("  • Precision (uncertainty): ε ≈ 1/√(shots)")
print("\nFor estimator with observable expectation values:")
print("  • Variance of estimate: Var ∝ 1/shots")
print("  • Standard deviation: σ_estimate ∝ 1/√(shots)")

print("\n📈 CONCRETE EXAMPLES:")
print("-" * 70)

# Example calculations
precisions = [0.1, 0.05, 0.01, 0.001]
base_shots = 1024

print(f"Base case: Precision = 0.1, Shots = {base_shots}\n")

for precision in precisions:
    # shots needed scales as 1/precision²
    shots_needed = int(base_shots * (0.1 / precision) ** 2)
    ratio = shots_needed / base_shots
    print(f"Precision = {precision:5.3f}")
    print(f"  → Shots needed: {shots_needed:,}")
    print(f"  → Ratio to base: {ratio:.1f}x")
    print()

print("\n⚡ KEY INSIGHTS:")
print("-" * 70)
print("1. SQUARE RELATIONSHIP:")
print("   • 2x better precision → 4x more shots")
print("   • 10x better precision → 100x more shots")
print("   • 100x better precision → 10,000x more shots")

print("\n2. DIMINISHING RETURNS:")
print("   • Each extra digit of precision is VERY expensive")
print("   • Going from 0.1 to 0.01 costs 100x shots")
print("   • Going from 0.01 to 0.001 costs another 100x shots")

print("\n3. PRACTICAL TRADE-OFFS:")
print("   • Default shots (1024-4096): ~1-2% precision")
print("   • High precision (0.1%): ~1,000,000 shots")
print("   • Chemical accuracy (0.01%): ~100,000,000 shots!")

print("\n🎯 EXAMPLE WITH ESTIMATOR:")
print("-" * 70)

# Create simple observable
observable = SparsePauliOp('Z')
qc_example = QuantumCircuit(1)
qc_example.h(0)  # |+⟩ state: ⟨Z⟩ = 0

print("Measuring ⟨Z⟩ for |+⟩ state (true value = 0):")
print()

# Simulate different shot counts

estimator = Estimator()
shot_counts = [100, 1000, 10000, 100000]

for shots in shot_counts:
    # Note: StatevectorEstimator is exact, so we simulate shot noise
    # In real hardware, you'd see this variance
    precision = 1.0 / np.sqrt(shots)
    print(f"Shots = {shots:6d} → Expected precision ≈ ±{precision:.4f}")

print("\n📋 FORMULA SUMMARY:")
print("=" * 70)
print("shots₂ = shots₁ × (precision₁ / precision₂)²")
print()
print("Or equivalently:")
print("precision₂ = precision₁ × √(shots₁ / shots₂)")

In [ ]:
# what does session.details() provide from qiskit_ibm_runtime.Session?
# --- IGNORE ---
# It provides metadata about the current session, including information such as session ID, creation time, expiration

# Section 1: Quantum Operations (16%)

## 1.1 Single-Qubit Gates - All Essential Gates

In [ ]:
# Probability of finding |0> after RY(π/3) with Bloch sphere visualization
# and theta/phi angles on the Bloch sphere
# Visualize states on the Bloch sphere with angles for RX, RY, RZ rotations

print("=" * 70)
print("Rotation Gate Analysis: RX(π/3), RY(π/3), RZ(π/3)")
print("=" * 70)
# In the geometric representation of qubit using theta and phi angles:
# the angle θ represents rotation from |0⟩ towards |1⟩ (latitude),
# while φ represents rotation around the Z-axis (longitude).
# formula of the state vector in terms of θ and φ:
# |ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)·sin(θ/2)|1⟩
# https://www.protocols.io/view/bsa-the-bloch-sphere-approach-as-a-geometrical-des-bp2l6dkkdvqe/v4

# Visualize RX, RY, RZ rotations on Bloch sphere with angle annotations

print("\n📊 Bloch Sphere Angle Convention:")
print("=" * 70)
print("  θ (theta) - Latitude/Polar angle:")
print("    • Range: 0 to π")
print("    • θ = 0   → North pole (|0⟩)")
print("    • θ = π/2 → Equator (superposition)")
print("    • θ = π   → South pole (|1⟩)")
print("\n  φ (phi) - Longitude/Azimuthal angle:")
print("    • Range: 0 to 2π")
print("    • φ = 0   → Positive X-axis")
print("    • φ = π/2 → Positive Y-axis")
print("    • φ = π   → Negative X-axis")
print("    • φ = 3π/2 → Negative Y-axis")
print("\n  State formula: |ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)·sin(θ/2)|1⟩")

print("\n" + "=" * 70)
print("📐 Geometric Effects of Rotation Gates on |0⟩:")
print("=" * 70)

# RY(θ) - Rotation around Y-axis
print("\n🔵 RY(θ) applied to |0⟩:")
print("  • Rotates in the X-Z plane (around Y-axis)")
print("  • Bloch sphere angle θ_bloch changes from 0 to θ")
print("  • Angle φ_bloch remains 0 (stays in X-Z plane)")
print("  • Geometric formula: |ψ⟩ = cos(θ/2)|0⟩ + sin(θ/2)|1⟩")

# RX(θ) - Rotation around X-axis  
print("\n🔴 RX(θ) applied to |0⟩:")
print("  • Rotates in the Y-Z plane (around X-axis)")
print("  • Bloch sphere angle θ_bloch changes from 0 to θ")
print("  • Angle φ_bloch = -π/2 (moves into Y-Z plane)")
print("  • Geometric formula: |ψ⟩ = cos(θ/2)|0⟩ + e^(i·(-π/2))·sin(θ/2)|1⟩")
print("  •                      = cos(θ/2)|0⟩ - i·sin(θ/2)|1⟩")

# RZ(θ) - Rotation around Z-axis
print("\n🟢 RZ(θ) applied to |0⟩:")
print("  • Rotates around Z-axis (longitude change)")
print("  • Bloch sphere angle θ_bloch remains 0 (stays at north pole)")
print("  • Angle φ_bloch changes by θ")
print("  • Geometric formula: |ψ⟩ = e^(-iθ/2)|0⟩ (global phase only)")
print("  • Note: RZ only affects |1⟩ component, so RZ(θ)|0⟩ = e^(-iθ/2)|0⟩")

print("\n" + "=" * 70)
print("📐 Geometric Effects of Rotation Gates on |1⟩:")
print("=" * 70)

# RY(θ) - Rotation around Y-axis applied to |1⟩
print("\n🔵 RY(θ) applied to |1⟩:")
print("  • Rotates in the X-Z plane (around Y-axis)")
print("  • Starting point: |1⟩ at south pole (θ_bloch = π)")
print("  • Bloch sphere angle θ_bloch changes from π to π-θ")
print("  • Angle φ_bloch = π (stays in X-Z plane, negative side)")
print("  • Geometric formula: |ψ⟩ = sin(θ/2)|0⟩ + cos(θ/2)|1⟩")

# RX(θ) - Rotation around X-axis applied to |1⟩
print("\n🔴 RX(θ) applied to |1⟩:")
print("  • Rotates in the Y-Z plane (around X-axis)")
print("  • Starting point: |1⟩ at south pole (θ_bloch = π)")
print("  • Bloch sphere angle θ_bloch changes from π to π-θ")
print("  • Angle φ_bloch = π/2 (moves into Y-Z plane)")
print("  • Geometric formula: |ψ⟩ = sin(θ/2)|0⟩ + e^(i·π/2)·cos(θ/2)|1⟩")
print("  •                      = sin(θ/2)|0⟩ + i·cos(θ/2)|1⟩")

# RZ(θ) - Rotation around Z-axis applied to |1⟩
print("\n🟢 RZ(θ) applied to |1⟩:")
print("  • Rotates around Z-axis (longitude change)")
print("  • Bloch sphere angle θ_bloch remains π (stays at south pole)")
print("  • Angle φ_bloch changes by θ")
print("  • Geometric formula: |ψ⟩ = e^(iθ/2)|1⟩")
print("  • Note: RZ(θ)|1⟩ = e^(iθ/2)|1⟩ (phase factor on |1⟩)")

print("\n" + "=" * 70)
print("Summary of Geometric Formulas:")
print("=" * 70)
print("Applied to |0⟩:")
print("  RX(θ)|0⟩ = cos(θ/2)|0⟩ - i·sin(θ/2)|1⟩            [φ = -π/2]")
print("  RY(θ)|0⟩ = cos(θ/2)|0⟩ + sin(θ/2)|1⟩              [φ = 0]")
print("  RZ(θ)|0⟩ = e^(-iθ/2)|0⟩                           [global phase]")
print("\nApplied to |1⟩:")
print("  RX(θ)|1⟩ = sin(θ/2)|0⟩ + i·cos(θ/2)|1⟩            [φ = π/2]")
print("  RY(θ)|1⟩ = sin(θ/2)|0⟩ + cos(θ/2)|1⟩              [φ = π]")
print("  RZ(θ)|1⟩ = e^(iθ/2)|1⟩                            [phase on |1⟩]")



In [ ]:
# 1.1 Single-Qubit Gates - Practice All Gates ONCE
qc = QuantumCircuit(3)

# Pauli Gates (X, Y, Z) - EXAM CRITICAL
qc.x(0)  # Bit flip: |0⟩→|1⟩, |1⟩→|0⟩
qc.y(0)  # Bit + phase flip
qc.z(0)  # Phase flip: |0⟩→|0⟩, |1⟩→-|1⟩

# Hadamard - Creates superposition
qc.h(1)  # H|0⟩ = |+⟩ = (|0⟩+|1⟩)/√2

# Phase Gates (S, T)
qc.s(1)  # S = √Z, S² = Z
qc.t(1)  # T = √S, T² = S, T⁴ = Z

# Rotation Gates (RX, RY, RZ)
qc.rx(np.pi/4, 2)  # Rotate around X-axis
qc.ry(np.pi/3, 2)  # Rotate around Y-axis
qc.rz(np.pi/6, 2)  # Rotate around Z-axis (phase gate)

# Phase gate (general)
qc.p(np.pi/8, 2)   # Add phase e^(iθ) to |1⟩

# Global phase gate (usually ignored)
qc.global_phase = np.pi/4

# Identity (no-op, but sometimes needed)
qc.id(2)

qc.draw('mpl')

In [ ]:
# EXAM CRITICAL: Gate Properties & Equivalences

# Self-inverse gates: X² = Y² = Z² = H² = I
qc = QuantumCircuit(1)
qc.x(0)
qc.x(0)  # XX = I (double X cancels out)
print(f"X² circuit depth: {qc.depth()}")  # Should be 2

# Gate equivalences
qc2 = QuantumCircuit(1)
qc2.h(0)
qc2.z(0)
qc2.h(0)  # HZH = X
print("HZH should equal X gate")

# Phase gate relationships: S² = Z, T² = S
qc3 = QuantumCircuit(1)
qc3.s(0)
qc3.s(0)  # S² = Z
qc3.z(0)  # Adding Z should give Z²=I
print(f"S² circuit size: {qc3.size()} gates")

# EXAM TRAP: Z only affects |1⟩
sv = Statevector.from_label('0')
sv_after = sv.evolve(Operator([[1, 0], [0, -1]]))  # Z gate
print(f"Z|0⟩ = {sv_after}")  # Should be |0⟩ (unchanged!)

# Gate commutativity - CRITICAL
# X and Z ANTICOMMUTE: XZ = -ZX
qc_xz = QuantumCircuit(1)
qc_xz.x(0)
qc_xz.z(0)

qc_zx = QuantumCircuit(1)
qc_zx.z(0)
qc_zx.x(0)
print("X and Z DON'T commute (anticommute with - sign)")

# But Z and S DO commute (both phase gates)
qc_zs = QuantumCircuit(1)
qc_zs.z(0)
qc_zs.s(0)

qc_sz = QuantumCircuit(1)
qc_sz.s(0)
qc_sz.z(0)
print("Z and S DO commute (both diagonal)")

## 1.2 Multi-Qubit Gates - All Two-Qubit Operations

In [ ]:
# 1.2 Multi-Qubit Gates - EXAM CRITICAL: Control BEFORE Target

qc = QuantumCircuit(4)

# CNOT/CX - Most important! Control first, target second
qc.cx(0, 1)  # q0 controls, q1 flips if q0=|1⟩

# Controlled-Z
qc.cz(0, 2)  # Symmetric! CZ(0,2) = CZ(2,0)

# SWAP - Exchange qubit states
qc.swap(1, 2)  # |10⟩ → |01⟩

# Controlled Phase
qc.cp(np.pi/4, 0, 3)  # Add phase if control=|1⟩

# Controlled-Y (less common)
qc.cy(1, 3)

# CH - Controlled Hadamard
qc.ch(2, 3)

# Multi-controlled gates
qc.ccx(0, 1, 2)  # Toffoli: flip target if BOTH controls are |1⟩
# qc.mct([0, 1, 2], 3)  # Multi-controlled Toffoli (3 controls)

qc.draw('mpl')

# EXAM PATTERN: What does CX(0,1) do to |11⟩?
# Answer: |11⟩ → |10⟩ (control=1, so flip target)

## 1.3 State Preparation - Creating Specific States

In [ ]:
# 1.3 State Preparation - All Methods

# Method 1: initialize() - EXAM CRITICAL
qc1 = QuantumCircuit(2)
statevector = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]  # Bell state
# expalin the arguements of initialize: first is the statevector, second is the qubits to apply to
# expalain the statevector format for 1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]: amplitudes for |00>, |01>, |10>, |11>
qc1.initialize(statevector, [0, 1])
qc1.draw('mpl')

# Method 2: prepare_state() - Similar to initialize
from qiskit.circuit.library import StatePreparation
qc2 = QuantumCircuit(2)
prep = StatePreparation(statevector)
qc2.append(prep, [0, 1])

# Method 3: Manual gates for common states
qc3 = QuantumCircuit(2)
# Bell state |Φ+⟩ = (|00⟩+|11⟩)/√2
qc3.h(0)
qc3.cx(0, 1)
print("Bell state created with H + CX")

# GHZ state (3 qubits)
qc4 = QuantumCircuit(3)
qc4.h(0)
qc4.cx(0, 1)
qc4.cx(0, 2)
# Result: (|000⟩+|111⟩)/√2

# W state - Use StatePreparation
w_state = [0, 1/np.sqrt(3), 1/np.sqrt(3), 0, 1/np.sqrt(3), 0, 0, 0]
qc5 = QuantumCircuit(3)
qc5.initialize(w_state, [0, 1, 2])

# EXAM PATTERN: "Create |+⟩ state"
qc6 = QuantumCircuit(1)
qc6.h(0)  # H|0⟩ = |+⟩
print("|+⟩ state with single H gate")

qc = QuantumCircuit(1)
qc.x(0)
sv = Statevector(qc)
print(f"State after X: {sv}")
print(f"Expected: [0, 1] = |1⟩")

# Section 2: Visualization (11%)

## 2.1 Circuit & State Visualization - All Methods

In [ ]:
from qiskit.visualization import plot_state_city, plot_state_qsphere, plot_state_hinton, plot_state_paulivec, plot_distribution

# 2.1 All Visualization Methods - Practice Each Once

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Circuit drawing - All output options with full parameters
qc.draw(
    output='mpl',           # Output format: 'text' (ASCII), 'mpl' (matplotlib), 'latex', 'latex_source'
    scale=1.0,              # Scaling factor for the circuit size
    filename=None,          # Path to save the figure (None means display only)
    style=None,             # Visual style: dict with colors/fonts or preset like 'iqp', 'clifford'
    interactive=False,      # Enable interactive navigation in matplotlib
    plot_barriers=True,     # Show barrier lines that separate circuit sections
    reverse_bits=False,     # Reverse qubit ordering (top to bottom)
    justify='left',         # Alignment of gates: 'left', 'right', 'none' (compress timing)
    vertical_compression='medium',  # Gate spacing: 'high' (compact), 'medium', 'low' (spacious)
    idle_wires=True,        # Show qubits with no gates applied
    with_layout=True,       # Display physical qubit layout for transpiled circuits
    fold=None,              # Wrap circuit at specified column width (None = no wrapping)
    ax=None,                # Existing matplotlib axes object to draw on
    initial_state=False,    # Show initial state labels (|0⟩) at circuit start
    cregbundle=True         # Bundle classical bits into single wire
)

# Histogram - Show measurement counts with full parameters
sampler = Sampler()
job = sampler.run([(qc,)], shots = 1000)
counts = job.result()[0].data.c.get_counts()

plot_histogram(
    counts,                 # Dictionary of measurement outcomes: {'00': count, '01': count} or list of dicts
    figsize=(7, 5),         # Figure dimensions in inches (width, height)
    color=None,             # Bar colors: single color string or list for multiple datasets
    number_to_keep=None,    # Display only top N most frequent outcomes
    sort='asc',             # Sorting: 'asc' (ascending), 'desc' (descending), 'hamming' (by Hamming weight), 'value', 'value_desc'
    target_string=None,     # Highlight specific bitstring with different color
    legend=None,            # Legend labels when plotting multiple datasets
    bar_labels=True,        # Display count values on top of bars
    title=None,             # Chart title text
    ax=None,                # Existing matplotlib axes to draw on
    filename=None           # Save path for the figure
)

# Bloch sphere - Visualize single-qubit states with full parameters
qc_single = QuantumCircuit(1)
qc_single.h(0)
sv = Statevector.from_instruction(qc_single)
plot_bloch_multivector(
    sv,                     # Quantum state: Statevector or density matrix
    title='',               # Figure title
    figsize=None,           # Figure size (width, height) in inches
    reverse_bits=False,     # Reverse qubit ordering in multi-qubit display
    filename=None,          # Save path for the figure
    font_size=None,         # Font size for labels
    title_font_size=None,   # Font size for title (separate from labels)
    title_pad=1             # Padding between title and figure in fraction of figure height
)

# State City Visualization - 3D bar chart showing real/imaginary amplitudes
plot_state_city(
    sv,                     # Quantum state: Statevector or density matrix
    title='',               # Figure title
    figsize=None,           # Figure size (width, height) in inches
    color=None,             # Two-element list: [real_part_color, imaginary_part_color]
    alpha=1,                # Transparency level: 0 (transparent) to 1 (opaque)
    ax_real=None,           # Existing matplotlib axes for real part
    ax_imag=None,           # Existing matplotlib axes for imaginary part
    filename=None           # Save path for the figure
)

# Q-sphere Visualization - Interactive sphere showing quantum state distribution
plot_state_qsphere(
    sv,                     # Quantum state: Statevector or density matrix
    figsize=None,           # Figure size (width, height) in inches
    ax=None,                # Existing matplotlib 3D axes to draw on
    show_state_labels=True, # Display basis state labels (|00⟩, |01⟩, etc.)
    show_state_phases=False,# Show phase values for each basis state
    use_degrees=False,      # Display phases in degrees instead of radians
    filename=None           # Save path for the figure
)

# Hinton Visualization - Square sizes represent amplitude magnitudes
plot_state_hinton(
    sv,                     # Quantum state: Statevector or density matrix
    title='',               # Figure title
    figsize=None,           # Figure size (width, height) in inches
    ax_real=None,           # Existing matplotlib axes for real part
    ax_imag=None,           # Existing matplotlib axes for imaginary part
    filename=None           # Save path for the figure
)

# Pauli Vector Visualization - Bar chart of expectation values for Pauli operators
plot_state_paulivec(
    sv,                     # Quantum state: Statevector or density matrix
    title='',               # Figure title
    figsize=None,           # Figure size (width, height) in inches
    color=None,             # Bar color(s) for Pauli expectation values
    ax=None,                # Existing matplotlib axes to draw on
    filename=None           # Save path for the figure
)

# EXAM CRITICAL: plot_histogram takes COUNTS dict
plot_histogram({'00': 500, '11': 500})  # Simple histogram with equal probabilities for |00⟩ and |11⟩

# Multiple histograms comparison
plot_histogram(
    [counts, {'00': 250, '01': 250, '10': 250, '11': 250}],  # List of count dictionaries to compare
    legend=['Bell State', 'Random'],  # Labels for each dataset in legend
    color=['blue', 'red']             # Colors for each dataset's bars
)
plot_distribution(counts)  # Alternative distribution plot

# Section 3: Circuit Creation and Composition (18%)
## 3.1 Basic Circuit Construction

In [ ]:
# 3.1 Circuit construction - All methods once

# Circuit composition with compose()
qc1 = QuantumCircuit(2)
qc1.h([0, 1])

qc2 = QuantumCircuit(2)
qc2.cx(0, 1)

# compose() - Add to existing circuit (IN-PLACE or NEW)
qc_combined = qc1.compose(qc2)  # Returns NEW circuit
qc1.compose(qc2, inplace=True)  # Modifies qc1

# Qubit mapping with qubits parameter
qc3 = QuantumCircuit(3)
qc3.compose(qc2, qubits=[1, 2])  # Apply qc2 to qubits 1,2 of qc3

# tensor() - Combine circuits side-by-side (⊗ operation)
qc_tensor = qc1.tensor(qc2)  # qc1 ⊗ qc2 = [qc1_qubits, qc2_qubits]
# EXAM TRAP: tensor() combines qubits, compose() adds gates sequentially

# Barriers - Prevent transpiler optimization across boundary
qc = QuantumCircuit(3)
qc.h(0)
qc.barrier()  # All qubits
qc.cx(0, 1)
qc.barrier([0, 1])  # Only qubits 0,1
qc.barrier(label='checkpoint')  # Named barrier

# Circuit properties vs methods
print(qc.num_qubits)  # PROPERTY - no ()
print(qc.depth())     # METHOD - has ()
print(qc.size())      # METHOD - gate count
print(qc.width())     # METHOD - total qubits + clbits

## 3.2 Parametric Circuits

In [ ]:
# 3.2 Parameters - All patterns once

from qiskit.circuit import Parameter, ParameterVector

# Single parameter
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.rx(theta, 0)

# Bind single value - Returns NEW circuit
qc_bound = qc.assign_parameters({theta: np.pi/4})

# ParameterVector - For multiple parameters
params = ParameterVector('θ', 3)
qc = QuantumCircuit(3)
qc.rx(params[0], 0)
qc.ry(params[1], 1)
qc.rz(params[2], 2)

# Bind vector - Dict or list
qc_bound = qc.assign_parameters({params: [np.pi/4, np.pi/2, np.pi]})
qc_bound_list = qc.assign_parameters([np.pi/4, np.pi/2, np.pi])  # Direct list

# Parameter arithmetic
alpha = Parameter('α')
beta = Parameter('β')
qc = QuantumCircuit(1)
qc.rx(2 * alpha + beta, 0)  # Expressions work
qc.ry(alpha ** 2, 0)

# EXAM CRITICAL: Primitives need binding
# Sampler/Estimator use parameter_values in run()
sampler = Sampler()
job = sampler.run([(qc, {alpha: 0.1, beta: 0.2})])  # Bind in run() - use actual parameters

# Get all parameters in circuit
print(qc.parameters)  # PROPERTY - returns ParameterView

## 3.3 Circuit Library and Classical Control

In [ ]:
# 3.3 Circuit Library - All standard circuits once

from qiskit.circuit.library import (QFTGate, grover_operator,
                                     efficient_su2, real_amplitudes, n_local)

# Standard algorithms - Use QFTGate instead of deprecated QFT
qft_gate = QFTGate(num_qubits=3)
qc = QuantumCircuit(3)
qc.append(qft_gate, [0, 1, 2])

# Grover operator - Use function instead of class
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover_op = grover_operator(oracle)
qc.append(grover_op, [0, 1])

# Variational forms for VQE/QAOA - Use functions instead of classes
qc = efficient_su2(num_qubits=3, reps=2)  # Ry-Rz layers
qc = real_amplitudes(num_qubits=3, reps=2)  # Only Ry gates
qc = n_local(num_qubits=3, rotation_blocks='ry', entanglement_blocks='cx', reps=2)

# Classical control - if_test pattern
from qiskit.circuit import ClassicalRegister
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure(0, 0)

# Apply gate based on classical bit
with qc.if_test((qc.clbits[0], 1)):  # If bit 0 == 1
    qc.x(1)

# while_loop and for_loop also available
# EXAM NOTE: Classical control not supported by all backends


## 3.4 Dynamic Circuits - Mid-Circuit Measurement & Control Flow

In [ ]:
# 3.4 Dynamic Circuits - All control flow patterns once

# if_test() - Modern conditional execution (PREFERRED)
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.measure(0, 0)

# Modern syntax - supports multiple gates in block
with qc.if_test((qc.clbits[0],1)) as else_:  # If c[0] == 1
    qc.x(1)
    qc.h(1)
with else_:
    qc.x(0)
    qc.z(1)  # Can have multiple gates!
print(qc.draw('mpl'))
# Legacy c_if() - single gate only (still tested!)
qc_legacy = QuantumCircuit(2, 2)
qc_legacy.h(0)
qc_legacy.measure(0, 0)
# qc_legacy.x(1).c_if(qc_legacy.clbits[0], 1)  # Single gate only

# while_loop() - Repeat until condition
qc_while = QuantumCircuit(2, 1)
with qc_while.while_loop((qc_while.clbits[0], 0)):  # While c[0] == 0
    qc_while.h(0)
    qc_while.measure(0, 0)

print(qc_while.draw('mpl'))


# for_loop() - Fixed iterations
qc_for = QuantumCircuit(1)
with qc_for.for_loop(range(5)):  # Repeat 5 times
    qc_for.h(0)
print(qc_for.draw('mpl'))


# With loop variable
with qc_for.for_loop(range(3)) as i:
    qc_for.rx(0.1 * i, 0)  # Different angle each iteration

# switch_case() - Multiple branches
qc_switch = QuantumCircuit(2, 2)
qc_switch.h(0)
qc_switch.h(1)
qc_switch.measure([0, 1], [0, 1])

with qc_switch.switch(qc_switch.clbits[0]) as case:
    with case(0):  # If c[0] == 0
        qc_switch.x(1)
    with case(1):  # If c[0] == 1
        qc_switch.z(1)
print(qc_switch.draw('mpl'))

# reset() - Return qubit to |0⟩ for reuse
qc_reset = QuantumCircuit(1, 2)
qc_reset.h(0)
qc_reset.measure(0, 0)
qc_reset.reset(0)  # Reset to |0⟩
qc_reset.x(0)      # Reuse qubit
qc_reset.measure(0, 1)

# EXAM CRITICAL: Dynamic circuits not supported by all backends!
# Check: backend.configuration().supported_features
# Aer simulator: ✅ Full support
# Real hardware: ⚠️ Check individually

# EXAM TRAP: if_test vs c_if
# if_test: Modern, with statement, multiple gates
# c_if: Legacy, method chaining, single gate only

# Section 4: Transpilation (15%)
## 4.1 All Transpilation Patterns

In [ ]:
# 4.1 Transpilation - Complete API Reference

from qiskit import transpile
from qiskit.transpiler import PassManager, generate_preset_pass_manager
from qiskit.transpiler.passes import Optimize1qGatesDecomposition
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.circuit.library import QFT
from qiskit import QuantumCircuit

print("=== TRANSPILATION COMPLETE API REFERENCE ===\n")

# === generate_preset_pass_manager() ===
print("=== generate_preset_pass_manager() - Full Control ===")

backend = FakeManilaV2()

# Full signature with all parameters
pm_full = generate_preset_pass_manager(
    optimization_level=2,              # Optimization level: 0-3 (2=default)
    backend=backend,                   # Target backend (V2 API)
    layout_method='sabre',             # 'sabre', 'dense', 'trivial'
    routing_method='sabre',            # 'sabre', 'basic', 'stochastic', 'lookahead'
    translation_method='translator',   # 'translator', 'synthesis'
    optimization_method='default',     # 'default', 'none'
    scheduling_method='alap',          # 'alap', 'asap', None
    seed_transpiler=None,              # Reproducibility seed
    coupling_map=None,                 # Override backend coupling (rare)
    basis_gates=None,                  # Override basis gates (rare)
    target=None,                       # Override target (advanced)
    hls_config=None,                   # High-level synthesis config
    initial_layout=None,               # Initial layout method
)

print(f"Generated PassManager with passes configured")

# === PassManager() ===
print("\n=== PassManager() - Custom Pipeline ===")

# Create custom pass manager with specific passes
custom_pm = PassManager(passes=[
    Optimize1qGatesDecomposition(),  # Example pass
])

print(f"Custom PassManager created")

# === TransformationPass - Custom Pass ===
print("\n=== TransformationPass - Custom Pass Creation ===")

from qiskit.transpiler import TransformationPass
from qiskit.dagcircuit import DAGCircuit

class CustomBarrierPass(TransformationPass):
    """Example custom transformation pass that adds barriers"""
    
    def __init__(self):
        super().__init__()
    
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        """Transform the DAG circuit by adding barriers"""
        # Custom transformation logic here
        # This is a simplified example
        return dag  # Return modified DAG

# Use custom pass
custom_pass = CustomBarrierPass()
custom_pm_with_custom = PassManager([custom_pass])
print("Custom TransformationPass created")

# === pm.run() ===
print("\n=== pm.run() - Apply Pass Manager ===")

# Create test circuit
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.append(QFT(3), [0, 1, 2])

print(f"Original circuit depth: {qc.depth()}")
print(f"Original circuit gates: {qc.count_ops()}")

# Apply pass manager - Returns NEW QuantumCircuit
transpiled_qc = pm_full.run(qc)

print(f"Transpiled circuit depth: {transpiled_qc.depth()}")
print(f"Transpiled circuit gates: {transpiled_qc.count_ops()}")

# === BACKEND TARGET API - V2 vs V1 ===
print("\n=== BACKEND TARGET API - V2 vs V1 Comparison ===\n")

# V2 (MODERN - USE THIS!)
print("V2 API (Properties - Modern):")
print(f"  backend.num_qubits: {backend.num_qubits}")
print(f"  backend.target.operation_names: {list(backend.target.operation_names)[:5]}...")

# Check if specific instruction is supported)
cx_supported = backend.target.instruction_supported('cx', qargs=(0,1))
print(f"  target.instruction_supported('cx', qargs=(0,1)): {cx_supported}")

# Build coupling map
coupling = backend.target.build_coupling_map()
print(f"  target.build_coupling_map(): {coupling.get_edges()[:3]}...")

# V1 (DEPRECATED - AVOID!)
print("\nV1 API (Configuration - Deprecated):")
print(f"  backend.configuration().n_qubits: {backend.configuration().n_qubits}")
print(f"  backend.configuration().basis_gates: {backend.configuration().basis_gates[:5]}...")
coupling_v1 = backend.configuration().coupling_map
print(f"  backend.configuration().coupling_map: {coupling_v1[:3] if coupling_v1 else None}...")

# === JOBS AND SESSIONS ===
print("\n=== JOBS AND SESSIONS - Complete API ===\n")

from qiskit.primitives import StatevectorSampler as Sampler

# Create simple circuit for testing
qc_simple = QuantumCircuit(1)
qc_simple.h(0)
qc_simple.measure_all()

sampler = Sampler()
job = sampler.run([qc_simple], shots=100)

# job.result() - BLOCKING (waits for completion)
# result = job.result(timeout=None)  # timeout in seconds, None = wait forever
# print(f"job.result(timeout=None): {result[0].data.meas.get_counts()}")

# job.status() - NON-BLOCKING (immediate return)
status = job.status()  # Returns JobStatus enum
print(f"job.status(): {status}")

# job.job_id() - Get unique job identifier for later retrieval
job_id = job.job_id()
print(f"job.job_id(): {job_id}")

# Session() - For iterative algorithms (e.g., VQE)
print("\nSession() - Context manager for iterative algorithms:")
print("Usage pattern:")
print("""
from qiskit_ibm_runtime import Session
with Session(backend=backend, max_time=None) as session:
    # All jobs in session share backend connection
    # Optimized for related, sequential jobs
    job1 = sampler.run([qc_simple], shots=100)
    job2 = sampler.run([qc_simple], shots=200)
""")

# Batch() - For parallel independent circuits
print("\nBatch() - Context manager for parallel jobs:")
print("Usage pattern:")
print("""
from qiskit_ibm_runtime import Batch
with Batch(backend=backend) as batch:
    # Jobs can be submitted in parallel
    # Optimized for independent circuits
    job_a = sampler.run([qc_simple], shots=100)
    job_b = sampler.run([qc_simple], shots=100)
""")

# === OPTIONS CONFIGURATION ===
print("\n=== OPTIONS CONFIGURATION - Complete Reference ===\n")

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Options are typically accessed through backend.options or passed to primitives
# Here we show the key properties and their defaults

print("Key Option Properties:")
print(f"  execution.shots: 4096 (default - NOT 1024!)")
print(f"  optimization_level: 2 (for Options), 1 (for transpile default)")
print(f"  resilience_level: 0 (0=none, 1=TREX, 2=ZNE)")
print(f"  simulator.seed_simulator: None (set for reproducibility)")

# Example of setting options (conceptual, as Options class usage varies)
print("\nExample usage with primitives:")
print("""
options = Options()
options.execution.shots = 2048
options.optimization_level = 3
options.resilience_level = 1  # TREX error mitigation
options.simulator.seed_simulator = 42
""")

# === RUNTIME SERVICE ===
print("\n=== QISKIT RUNTIME SERVICE - Complete API ===\n")

print("Runtime Service Methods (requires IBM Quantum credentials):")

# save_account() - One-time setup
print("\n1. save_account() - One-time credential setup:")
print("""
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(
    channel="ibm_quantum",  # or "ibm_cloud"
    token="YOUR_API_TOKEN",  # From IBM Quantum dashboard
    instance="ibm-q/open/main"  # For cloud instances
)
""")

# QiskitRuntimeService() - Connect to service
print("\n2. QiskitRuntimeService() - Connect to IBM Quantum:")
print("""
service = QiskitRuntimeService(
    channel="ibm_quantum",  # or "ibm_cloud"
    token=None  # Uses saved account if None
)
""")

# backends() - Get available backends
print("\n3. backends() - Get list of available backends:")
print("""
# Get all backends
all_backends = service.backends()

# Filter backends
simulator_backends = service.backends(
    simulator=True,
    operational=True,
    min_num_qubits=5
)
""")

# backend() - Get specific backend by name
print("\n4. backend(name) - Get specific backend:")
print("""
specific_backend = service.backend("ibm_brisbane")
print(specific_backend.name)
""")

# least_busy() - Auto-select best backend
print("\n5. least_busy() - Auto-select shortest queue:")
print("""
best_backend = service.least_busy(
    min_num_qubits=5,
    simulator=False,
    operational=True
)
print(f"Least busy: {best_backend.name}")
""")

print("\n=== EXAM PATTERNS ===")
print("✓ generate_preset_pass_manager() for full transpilation control")
print("✓ PassManager() for custom optimization pipelines")
print("✓ pm.run(circuit) returns NEW QuantumCircuit")
print("✓ V2 API uses properties (backend.num_qubits), V1 uses configuration()")
print("✓ job.result() BLOCKS, job.status() is NON-BLOCKING")
print("✓ Session for iterative algorithms, Batch for parallel jobs")
print("✓ Options defaults: shots=4096, optimization_level=2, resilience_level=0")
print("✓ Runtime: save_account() once, then service.backends(), backend(), least_busy()")

# Section 5: Sampler Primitive (12%)
## 5.1 All Sampler Patterns - PUB Format and Results

In [ ]:
# 5.1 Sampler - PUB format and all result patterns

from qiskit.primitives import StatevectorSampler as Sampler

# Basic Sampler usage - PUB format [circuit]
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = Sampler()
pub = [(qc,)]
job = sampler.run(pub, shots=1024)  # PUB: [circuit] with trailing comma optional
result = job.result()

# EXAM CRITICAL: Result extraction pattern
# result[PUB_index].data.REGISTER_NAME.get_counts()
counts = result[0].data.meas.get_counts()  # 'meas' is default register from measure_all()
print(counts)  # {'00': ~512, '11': ~512}

# Custom register names
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(0, 1)
qc2.measure([0, 1], [0, 1])  # Register named by ClassicalRegister
pub2 = [(qc2,)]
job2 = sampler.run(pub2, shots = 1024)
result2 = job2.result()
counts2 = result2[0].data.c.get_counts()  # Default name is 'c' for manual measure

# Parametric circuits with Sampler
theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)
qc_param.measure_all()

# PUB format: (circuit, parameter_values) or (circuit, {param: value})
param1_pub = (qc_param, [np.pi/4])
job = sampler.run([param1_pub], shots=1000)
param2_pub = (qc_param, [np.pi/5])
job3 = sampler.run([param2_pub], shots=1000)

# Multiple circuits in one job
# For parametric circuits, must provide parameter values in PUB format
job = sampler.run([qc, qc2, (qc_param, [np.pi/4])], shots=512)  # Mix of parameterized and non-parameterized
result = job.result()
counts0 = result[0].data.meas.get_counts()
counts1 = result[1].data.c.get_counts()
counts2 = result[2].data.meas.get_counts()

# EXAM TRAP: Sampler needs measure gates! No measurement = Error

# Section 6: Estimator Primitive (12%)
## 6.1 All Estimator Patterns - Observables and Expectation Values

In [ ]:
# 6.1 Estimator - Observables and expectation values

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

# Basic Estimator - Expectation value of observable
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Observable - SparsePauliOp
observable = SparsePauliOp.from_list([('ZZ', 1.0), ('XX', 0.5)])
# Alternative: SparsePauliOp(['ZZ', 'XX'], coeffs=[1.0, 0.5])

estimator = Estimator()
job = estimator.run([(qc, observable)])  # PUB: (circuit, observable)
result = job.result()

# EXAM CRITICAL: Result extraction - .data.evs
expectation_value = result[0].data.evs  # Expectation values array
print(f"<ψ|H|ψ> = {expectation_value}")

# Multiple observables for same circuit
obs1 = SparsePauliOp('ZZ')
obs2 = SparsePauliOp('XX')
obs3 = SparsePauliOp('YY')

job = estimator.run([(qc, obs1), (qc, obs2), (qc, obs3)])
result = job.result()
ev1 = result[0].data.evs
ev2 = result[1].data.evs
ev3 = result[2].data.evs

# Parametric circuits with Estimator
theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)

observable = SparsePauliOp('Z')
# PUB: (circuit, observable, parameter_values)
job = estimator.run([(qc_param, observable, [np.pi/4])])
job2 = estimator.run([(qc_param, observable, {theta: np.pi/4})])

# Hamiltonian simulation - Common use case
H = SparsePauliOp.from_list([
    ('II', -1.0),
    ('ZZ', 0.5),
    ('XX', 0.3)
])
energy = estimator.run([(qc, H)]).result()[0].data.evs

# EXAM TRAP: Estimator doesn't need measure gates (works on statevector)

# Section 7: Results Extraction (10%)
## 7.1 All Result Access Patterns - RIDMG Method

In [ ]:
# 7.1 Results - RIDMG extraction pattern (Result → Index → Data → Method → Get)

# SAMPLER RESULTS
sampler = StatevectorSampler()
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

job = sampler.run([qc], shots=1000)
result = job.result()

# RIDMG chain for Sampler
# R: result
# I: [index] - PUB index
# D: .data
# M: .REGISTER_NAME (meas, c, etc.)
# G: .get_counts() or .get_bitstrings()
counts = result[0].data.meas.get_counts()
bitstrings = result[0].data.meas.get_bitstrings()  # List of bitstrings

# Multiple registers
qc_multi = QuantumCircuit(2, 2)
qc_multi.add_register(ClassicalRegister(2, 'aux'))
qc_multi.h(0)
qc_multi.measure(0, 0)
qc_multi.measure(1, 2)  # To 'aux' register

job = sampler.run([qc_multi])
result = job.result()
counts_c = result[0].data.c.get_counts()
counts_aux = result[0].data.aux.get_counts()

# ESTIMATOR RESULTS  
estimator = StatevectorEstimator()
observable = SparsePauliOp('ZZ')

qc_estimator = QuantumCircuit(2)
qc_estimator.h(0)
qc_estimator.cx(0, 1)
job = estimator.run([(qc_estimator, observable)])
result = job.result()


# RIDE chain for Estimator (no Method, direct to .evs)
# R: result
# I: [index]
# D: .data
# E: .evs (expectation values)
expectation_value = result[0].data.evs

# Standard deviation also available
std_dev = result[0].data.stds

# EXAM CRITICAL: Know the pattern by heart
# Sampler: result[i].data.REGISTER.get_counts()
# Estimator: result[i].data.evs

# Section 8: OpenQASM (6%)
## 8.1 All OpenQASM Operations - Import/Export

In [ ]:
# 8.1 OpenQASM - All import/export patterns

# Export circuit to QASM 2.0
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

from qiskit import qasm2, qasm3
qasm_string = qasm2.dumps(qc)  # Returns QASM 2.0 string
print(qasm_string)

# EXAM CRITICAL: Import from QASM string - STATIC METHOD!
qasm_code = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];
"""

# QuantumCircuit.from_qasm_str() is STATIC, not instance method
qc_imported = QuantumCircuit.from_qasm_str(qasm_code)

# Import from QASM file
# qc_file = QuantumCircuit.from_qasm_file('circuit.qasm')
# OpenQASM 3.0 - Newer version with more features
qasm3_string = qasm3.dumps(qc)  # Returns QASM 3.0 string

# Import QASM 3.0 - Use qiskit.qasm3
from qiskit import qasm3
qasm3_code = """
OPENQASM 3;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
"""
qc_qasm3 = qasm3.loads(qasm3_code)

# Custom QASM3 Exporter - Advanced control
# Default exporter includes standard gates
qasm3_default = qasm3.dumps(qc)  # Includes "stdgates.inc"

# Custom exporter with ALL parameters documented
exporter = qasm3.Exporter(
    includes=[],                    # List of include files (e.g., ['stdgates.inc'])
    basis_gates=None,               # List of basis gates to use (None = use circuit's gates)
    disable_constants=True,         # Disable constant folding for parameters
    alias_classical_registers=None, # Allow aliasing of classical registers (None = auto)
    allow_aliasing=None,            # Deprecated: use alias_classical_registers
    indent='  ',                    # Indentation string for code blocks
    experimental=None               # Enable experimental features (internal use)
)
exporter = qasm3.Exporter(includes = [], disable_constants=True)
qasm3_custom = exporter.dumps(qc)
print("Custom export (no includes):")
print(qasm3_custom)

# Custom exporter with specific includes
exporter_custom_includes = qasm3.Exporter(
    includes=['stdgates.inc'],  # Specific includes
    disable_constants=False,    # Enable constant folding
)

# Advanced exporter options
exporter_advanced = qasm3.Exporter(
    includes=[],
    basis_gates=['u', 'cx'],   # Only export these gates
    disable_constants=True,
    allow_aliasing=False,      # No register aliasing
)

# EXAM NOTE: Custom exporters useful for:
# - Hardware-specific gate sets
# - Minimal QASM output
# - Custom gate definitions

# EXAM TRAP: from_qasm_str() is STATIC method on QuantumCircuit class
# WRONG: qc.from_qasm_str(qasm_code)  ❌
# RIGHT: QuantumCircuit.from_qasm_str(qasm_code)  ✓

# Section 9: Quantum Information (3%)
## 9.1 Clifford Circuits and Operator Class

In [ ]:
# 9.1 Clifford Circuits - Most common exam trap!

from qiskit.quantum_info import Clifford, Operator

# EXAM CRITICAL: Clifford gates = {H, S, S†, CNOT, X, Y, Z}
# T gate is NOT Clifford! (Most tested trap)

# ✅ Valid Clifford circuit
qc_clifford = QuantumCircuit(2)
qc_clifford.h(0)
qc_clifford.s(1)
qc_clifford.cx(0, 1)
qc_clifford.x(0)
cliff = Clifford(qc_clifford)  # Works!

# ❌ Invalid - T gate is NOT Clifford
qc_with_t = QuantumCircuit(1)
qc_with_t.h(0)
qc_with_t.t(0)  # Adding T breaks Clifford property
# cliff_fail = Clifford(qc_with_t)  # This would FAIL!

# Clifford operations
cliff1 = Clifford.from_circuit(qc_clifford)
# Create a separate circuit for cliff2
qc_cx = QuantumCircuit(2)
qc_cx.cx(0, 1)
cliff2 = Clifford.from_circuit(qc_cx)

# Check equivalence
print(f"Clifford equality: {cliff1 == cliff2}")

# Convert back to circuit
qc_back = cliff1.to_circuit()

# Compose Cliffords
composed = cliff1.compose(cliff2)

# 9.1 Operator Class - Use .equiv() not ==

# Create operators from circuits
qc1 = QuantumCircuit(1)
qc1.h(0)
op1 = Operator(qc1)

qc2 = QuantumCircuit(1)
qc2.h(0)
qc2.z(0)  # Global phase difference
op2 = Operator(qc2)

# EXAM TRAP: Use .equiv() for physical equivalence
print(f"op1 == op2: {op1 == op2}")  # False (strict matrix)
print(f"op1.equiv(op2): {op1.equiv(op2)}")  # True (ignores global phase)

# Compose order (counterintuitive!)
# op1.compose(op2) applies op2 FIRST, then op1
h_op = Operator.from_label('H')
x_op = Operator.from_label('X')
result = x_op.compose(h_op)  # H first, then X

# Get unitary matrix
print(f"Matrix shape: {op1.data.shape}")

## 9.2 Statevector and DensityMatrix

In [ ]:
# 9.2 Statevector (pure) vs DensityMatrix (mixed)

from qiskit.quantum_info import Statevector, DensityMatrix

# Statevector - Pure states only
sv_0 = Statevector.from_label('0')  # |0⟩
sv_plus = Statevector.from_label('+')  # |+⟩ = (|0⟩ + |1⟩)/√2

# Create from circuit
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)
bell_state = Statevector.from_instruction(qc_bell)

# Get probabilities
probs = bell_state.probabilities()
print(f"Bell state probs: {probs}")  # [0.5, 0, 0, 0.5]

# Evolve with operator
evolved = sv_0.evolve(Operator.from_label('X'))  # X|0⟩ = |1⟩

# DensityMatrix - Can represent pure AND mixed states
# Pure state from Statevector
rho_pure = DensityMatrix(sv_plus)
print(f"Pure state purity: {rho_pure.purity()}")  # 1.0

# Mixed state (classical probability mixture)
# Maximally mixed state: ρ = I/2
mixed = DensityMatrix(np.eye(2) / 2)
print(f"Mixed state purity: {mixed.purity()}")  # 0.5

# EXAM CRITICAL: Superposition ≠ Mixture!
# |+⟩ is SUPERPOSITION (pure, purity=1, shows interference)
# 50% |0⟩ + 50% |1⟩ mixture is MIXED (purity=0.5, no interference)

# Check purity: Tr(ρ²)
# Pure state: purity = 1
# Mixed state: purity < 1

# Convert pure Statevector to DensityMatrix
dm_from_sv = DensityMatrix(bell_state)
print(f"Bell state as DM purity: {dm_from_sv.purity()}")  # 1.0 (still pure)

## 9.3 Fidelity and Quantum Channels

In [ ]:
# 9.3 Fidelity - ALWAYS between [0, 1]

from qiskit.quantum_info import state_fidelity, process_fidelity, average_gate_fidelity

# State fidelity - Compare two quantum states
state1 = Statevector.from_label('0')
state2 = Statevector.from_label('+')
state3 = Statevector.from_label('1')

fid_01 = state_fidelity(state1, state2)
print(f"Fidelity |0⟩ vs |+⟩: {fid_01}")  # 0.5

fid_same = state_fidelity(state1, state1)
print(f"Fidelity same state: {fid_same}")  # 1.0

fid_ortho = state_fidelity(state1, state3)
print(f"Fidelity orthogonal: {fid_ortho}")  # 0.0

# EXAM TRAP: Fidelity is ALWAYS [0, 1], never exceeds 1!
# 1 = identical, 0 = orthogonal

# Process fidelity - Compare operations/gates
ideal_gate = Operator.from_label('X')
noisy_gate = Operator.from_label('X')  # Assume this is noisy

proc_fid = process_fidelity(noisy_gate, ideal_gate)
print(f"Process fidelity: {proc_fid}")

# Average gate fidelity - Standard metric for gate quality
avg_fid = average_gate_fidelity(noisy_gate, ideal_gate)
print(f"Average gate fidelity: {avg_fid}")

# Fidelity interpretation:
# 0.99+ = Excellent
# 0.95-0.99 = Good
# 0.90-0.95 = Moderate
# < 0.90 = Poor

# 9.3 Quantum Channels - Three equivalent representations

from qiskit.quantum_info import Kraus, SuperOp, Choi

# Kraus representation - Best for physics intuition
# Describes how noise acts: ρ → Σₖ Kₖ ρ Kₖ†

# SuperOp representation - Best for math
# Matrix on vectorized density matrix

# Choi representation - Best for tomography
# Channel applied to maximally entangled state

# They're all equivalent - can convert between them
# Example: Bit flip channel
p = 0.1  # Error probability
K0 = np.sqrt(1 - p) * np.eye(2)
K1 = np.sqrt(p) * np.array([[0, 1], [1, 0]])  # X gate


kraus_channel = Kraus([K0, K1])
superop_channel = SuperOp(kraus_channel)
choi_channel = Choi(kraus_channel)

# All represent the same physical channel
# Use Kraus for understanding, SuperOp for calculation, Choi for analysis

# EXAMPLE: Applying quantum channels to circuits
print("\n" + "=" * 70)
print("APPLYING QUANTUM CHANNELS TO CIRCUITS")
print("=" * 70)

# Create a simple quantum state
qc_channel_test = QuantumCircuit(1)
qc_channel_test.h(0)  # Create |+⟩ state

# Get initial state as density matrix
initial_state = DensityMatrix.from_instruction(qc_channel_test)
print(f"\nInitial state |+⟩:")
print(f"Density matrix:\n{initial_state.data}")
print(f"Purity: {initial_state.purity():.4f} (pure state)")

# Apply bit flip channel (10% chance of X error)
noisy_state = initial_state.evolve(kraus_channel)
print(f"\nAfter bit flip channel (p=0.1):")
print(f"Density matrix:\n{noisy_state.data}")
print(f"Purity: {noisy_state.purity():.4f} (mixed state due to noise)")

# Compare with using SuperOp (same result)
noisy_state_superop = initial_state.evolve(superop_channel)
print(f"\nUsing SuperOp (should be same):")
print(f"States equal: {np.allclose(noisy_state.data, noisy_state_superop.data)}")

# Example: Depolarizing channel (more realistic noise)
p_depol = 0.05
K0_depol = np.sqrt(1 - 3*p_depol/4) * np.eye(2)
K1_depol = np.sqrt(p_depol/4) * np.array([[0, 1], [1, 0]])  # X
K2_depol = np.sqrt(p_depol/4) * np.array([[0, -1j], [1j, 0]])  # Y
K3_depol = np.sqrt(p_depol/4) * np.array([[1, 0], [0, -1]])  # Z

depol_channel = Kraus([K0_depol, K1_depol, K2_depol, K3_depol])
depol_state = initial_state.evolve(depol_channel)

print(f"\nAfter depolarizing channel (p=0.05):")
print(f"Density matrix:\n{depol_state.data}")
print(f"Purity: {depol_state.purity():.4f}")

# EXAM TIP: Channels are applied to density matrices, not statevectors
# For pure states, convert to DensityMatrix first, then apply channel

# 🎯 EXAM CRITICAL TRAPS - Final Checklist

## Properties vs Methods
- `qc.num_qubits` (PROPERTY - no parentheses)
- `qc.depth()` (METHOD - has parentheses)
- `qc.parameters` (PROPERTY - returns ParameterView)

## Static Methods
- `QuantumCircuit.from_qasm_str(qasm_string)` - MUST use class name
- NOT `qc.from_qasm_str()` ❌

## Result Extraction Patterns
- **Sampler**: `result[i].data.REGISTER_NAME.get_counts()`
- **Estimator**: `result[i].data.evs`
- Register names: 'meas' (measure_all), 'c' (manual measure)

## Transpilation
- `transpile()` returns NEW circuit (doesn't modify original)
- Default optimization_level is 2

## Composition
- `compose()` - Sequential (adds gates in time)
- `tensor()` - Parallel (combines qubits side-by-side)

## Sampler Requirements
- MUST have measure gates
- Without measurement → Error

## Estimator Features
- Works on statevector (no measurement needed)
- Takes SparsePauliOp observables

## Parameter Binding
- `assign_parameters()` returns NEW circuit
- Can bind in primitive run: `sampler.run([(qc, parameter_values)])`

## Section 9: Quantum Information Traps
- **T gate is NOT Clifford!** (Most common trap) ⚠️
- Clifford gates: {H, S, S†, CNOT, X, Y, Z}
- Use `Operator.equiv()` NOT `==` for circuit equivalence
- `op1.compose(op2)` applies op2 FIRST (right-to-left)
- **Fidelity range: ALWAYS [0, 1]** - never exceeds 1
- Statevector = Pure states only (purity = 1)
- DensityMatrix = Pure + Mixed states (purity ≤ 1)
- **Superposition ≠ Mixture!** |+⟩ is pure, not mixed
- Three channel representations: Kraus, SuperOp, Choi (all equivalent)

---
✅ All 9 sections covered with runnable code
✅ Each concept appears exactly once  
✅ Direct practice-focused examples